# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
# Final Model Download URL - https://drive.google.com/file/d/1ARIfphgOL68I-P4eHvT0pWtsbQUEsV1N/view?usp=sharing
# File Name - model-00050-1.52472-0.33032-1.52612-0.34000.h5

In [2]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 30 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [6,8,10,11,12,13,14,16,18,20,22,24] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size  # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,12,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    lx, ly, lz = image.shape
                    # Cropping
                    image = image[lx//4:-lx//4,ly//4:-ly//4]
                    image  = imresize(image, [120, 120]) #resize image
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    batch_data[folder,idx,:,:,0] = image[:, : , 0] - np.min(image[:, : , 0])/np.max(image[:, : , 0]) - np.min(image[:, : , 0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:, : , 1] - np.min(image[:, : , 1])/np.max(image[:, : , 1]) - np.min(image[:, : , 1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:, : , 2] - np.min(image[:, : , 2])/np.max(image[:, : , 2]) - np.min(image[:, : , 2]) #normalise and feed in the image
                        
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        # write the code for the remaining data points which are left after full batches
        remaining_sequences = int(len(folder_list)%batch_size)
        batch_data = np.zeros((remaining_sequences,12,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((remaining_sequences,5)) # batch_labels is the one hot representation of the output
        for folder in range(remaining_sequences): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + num_batches*batch_size].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + num_batches*batch_size].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                lx, ly, lz = image.shape
                # Cropping
                image = image[lx//4:-lx//4,ly//4:-ly//4]               
                image  = imresize(image, [120, 120])
                batch_data[folder,idx,:,:,0] = image[:, : , 0] - np.min(image[:, : , 0])/np.max(image[:, : , 0]) - np.min(image[:, : , 0]) #normalise and feed in the image
                batch_data[folder,idx,:,:,1] = image[:, : , 1] - np.min(image[:, : , 1])/np.max(image[:, : , 1]) - np.min(image[:, : , 1]) #normalise and feed in the image
                batch_data[folder,idx,:,:,2] = image[:, : , 2] - np.min(image[:, : , 2])/np.max(image[:, : , 2]) - np.min(image[:, : , 2]) #normalise and feed in the image

            batch_labels[folder, int(t[folder + num_batches*batch_size].strip().split(';')[2])] = 1
        yield batch_data, batch_labels 

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling3D, ZeroPadding3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications import ResNet50, VGG16
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.regularizers import l2

#write your model here
num_classes = 5

# Conv2D + RNN Transfer Learning
# VGG16
input_layer = Input(shape=(12, 120, 120, 3))
base_model = VGG16(include_top=False, weights='imagenet')
x = TimeDistributed(base_model)(input_layer); 
# x = TimeDistributed(GlobalAveragePooling2D())(x)
x = TimeDistributed(Flatten())(x)
x = GRU(64, return_sequences=False)(x)
x = Dense(num_classes, activation='softmax')(x)
#x = Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01))(x)
#x = Dropout(0.50)(x)
model = Model(inputs=input_layer, outputs=x)

# Conv2D + RNN
# model = Sequential()
# model.add(TimeDistributed(Conv2D(64,(3,3), activation='relu'), input_shape=(12, 120, 120, 3)))
# model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))
# model.add(TimeDistributed(Conv2D(128, (4,4), activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
# model.add(TimeDistributed(Conv2D(256, (4,4), activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# model.add(TimeDistributed(Flatten()))
# model.add(GRU(16, return_sequences=False))
# model.add(Dense(num_classes, activation='softmax'))
          

# Conv3D Model
## input layer
# model = Sequential()
# model.add(ZeroPadding3D(1, input_shape=(12, 120, 120, 3)))
# model.add(Conv3D(64,(3,3,3), activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# # model.add(Dropout(0.25))
# model.add(Conv3D(128,(3,3,3), activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# # model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(512,kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.50))
# model.add(Dense(num_classes, activation='softmax'))




Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 12, 120, 120, 3)   0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 12, 3, 3, 512)     14714688  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 12, 4608)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                897216    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 15,612,229
Trainable params: 15,612,229
Non-trainable params: 0
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
   steps_per_epoch = int(num_train_sequences/batch_size)
else:
   steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
   validation_steps = int(num_val_sequences/batch_size)
else:
   validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
Source path =  Project_data/train ; batch size = 30
Source path =  Project_data/val ; batch size = 30
23/23 [==============================] - 43s 2s/step - loss: 1.7586 - categorical_accuracy: 0.2181 - val_loss: 2.1430 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2018-12-3010_20_34.541693/model-00001-1.77385-0.20060-2.14302-0.21000.h5
Epoch 2/50
23/23 [==============================] - 32s 1s/step - loss: 1.7184 - categorical_accuracy: 0.2192 - val_loss: 2.8520 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2018-12-3010_20_34.541693/model-00002-1.69121-0.22775-2.85205-0.22000.h5
Epoch 3/50
23/23 [==============================] - 32s 1s/step - loss: 1.6937 - categorical_accuracy: 0.2250 - val_loss: 1.6010 - val_categorical_accuracy: 0.2800

Epoch 00003: saving model to model_init_2018-12-3010_20_34.541693/model-00003-1.68786-0.23379-1.60101-0.28000.h5
Epoch 4/50
23/23 [==============================] - 32s 1s/step 

23/23 [==============================] - 33s 1s/step - loss: 1.5176 - categorical_accuracy: 0.3319 - val_loss: 1.5258 - val_categorical_accuracy: 0.3300

Epoch 00030: saving model to model_init_2018-12-3010_20_34.541693/model-00030-1.52548-0.33183-1.52577-0.33000.h5
Epoch 31/50
23/23 [==============================] - 33s 1s/step - loss: 1.5084 - categorical_accuracy: 0.3429 - val_loss: 1.5255 - val_categorical_accuracy: 0.3400

Epoch 00031: saving model to model_init_2018-12-3010_20_34.541693/model-00031-1.52557-0.33032-1.52547-0.34000.h5
Epoch 32/50
23/23 [==============================] - 33s 1s/step - loss: 1.5065 - categorical_accuracy: 0.3569 - val_loss: 1.5248 - val_categorical_accuracy: 0.3400

Epoch 00032: saving model to model_init_2018-12-3010_20_34.541693/model-00032-1.52554-0.33183-1.52475-0.34000.h5
Epoch 33/50
23/23 [==============================] - 33s 1s/step - loss: 1.5200 - categorical_accuracy: 0.3458 - val_loss: 1.5241 - val_categorical_accuracy: 0.3400

Epoch 000